In [222]:
import pandas as pd
import os
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [30, 20]


In [223]:
PATH = "data"
INPUT_FILE = "og_data.csv"
OUTPUT_FILE = "ready_data.csv"

In [224]:
df = pd.read_csv(os.path.join(PATH,INPUT_FILE), index_col=0)
df = df.reset_index()
df = df.drop("index", axis=1)
df.head()


,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [225]:
print(f"Rows: {len(df)}, Cols: {len(df.columns)}")

Rows: 84548, Cols: 21


In [226]:
# Lets first only ue zip code and not include other information about the excact location
# + lets only use at time of sale values instead of at present values
# + "EASE-MENT" col seems to only contain empty values
print("Orginal columns:", df.columns.values)
redundant_cols = ['SALE DATE', 'ADDRESS', 'BUILDING CLASS AT PRESENT', 'TAX CLASS AT PRESENT', 'NEIGHBORHOOD', "BUILDING CLASS CATEGORY", "EASE-MENT", "APARTMENT NUMBER"]
cleaned_df = df.drop(redundant_cols, axis=1)
print("\n Columns we need:", cleaned_df.columns.values)
cleaned_df.head()

Orginal columns: ['BOROUGH' 'NEIGHBORHOOD' 'BUILDING CLASS CATEGORY' 'TAX CLASS AT PRESENT'
 'BLOCK' 'LOT' 'EASE-MENT' 'BUILDING CLASS AT PRESENT' 'ADDRESS'
 'APARTMENT NUMBER' 'ZIP CODE' 'RESIDENTIAL UNITS' 'COMMERCIAL UNITS'
 'TOTAL UNITS' 'LAND SQUARE FEET' 'GROSS SQUARE FEET' 'YEAR BUILT'
 'TAX CLASS AT TIME OF SALE' 'BUILDING CLASS AT TIME OF SALE' 'SALE PRICE'
 'SALE DATE']

 Columns we need: ['BOROUGH' 'BLOCK' 'LOT' 'ZIP CODE' 'RESIDENTIAL UNITS' 'COMMERCIAL UNITS'
 'TOTAL UNITS' 'LAND SQUARE FEET' 'GROSS SQUARE FEET' 'YEAR BUILT'
 'TAX CLASS AT TIME OF SALE' 'BUILDING CLASS AT TIME OF SALE' 'SALE PRICE']


,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE
0,1,392,6,10009,5,0,5,1633,6440,1900,2,C2,6625000
1,1,399,26,10009,28,3,31,4616,18690,1900,2,C7,-
2,1,399,39,10009,16,1,17,2212,7803,1900,2,C7,-
3,1,402,21,10009,10,0,10,2272,6794,1913,2,C4,3936272
4,1,404,55,10009,6,0,6,2369,4615,1900,2,C2,8000000


In [227]:
# Like mentioned in kaggle "Many sales occur with a nonsensically small dollar amount: $0 most commonly"
# Delete those

len_before = len(cleaned_df)
cleaned_df = cleaned_df[cleaned_df["SALE PRICE"] != " -  "]
cleaned_df = cleaned_df[cleaned_df["LAND SQUARE FEET"] != " -  "]
cleaned_df = cleaned_df[cleaned_df["GROSS SQUARE FEET"] != " -  "]


len_after = len(cleaned_df)

print(f"Number of deleted rows: {len_before - len_after}")

Number of deleted rows: 36304


In [228]:
# Turn "BUILDING CLASS AT TIME OF SALE" column to dummy variable
# Orginally they are listed with two char identifier. For example "C7"
# We want them to be just number starting from 0.

classes = cleaned_df["BUILDING CLASS AT TIME OF SALE"].unique()
conversion_dict = {item:idx for idx, item in enumerate(classes)} 
def convert_class(row):
    ret = conversion_dict[row]
    return ret

cleaned_df["BUILDING CLASS AT TIME OF SALE"] = cleaned_df["BUILDING CLASS AT TIME OF SALE"].apply(convert_class)
cleaned_df.head()


,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE
0,1,392,6,10009,5,0,5,1633,6440,1900,2,0,6625000
3,1,402,21,10009,10,0,10,2272,6794,1913,2,1,3936272
4,1,404,55,10009,6,0,6,2369,4615,1900,2,0,8000000
6,1,406,32,10009,8,0,8,1750,4226,1920,2,1,3192840
9,1,387,153,10009,24,0,24,4489,18523,1920,2,2,16232000


In [229]:
# No missing values in other columns
print("Missing values per col:")
print(cleaned_df.isnull().sum(axis = 0))

print()
print("Columnn and and sum of zero values:")
for col in cleaned_df.columns.values:
    print(f"{col} : {sum(cleaned_df[col] == 0)}")

cleaned_df = cleaned_df[cleaned_df["YEAR BUILT"] != 0]
cleaned_df = cleaned_df[cleaned_df["ZIP CODE"] != 0]

# Unfortunately there are allmost 20k datapoinst with total units == 0.
# I feel that house must have at least one unit but for now lets keep thos

Missing values per col:
BOROUGH                           0
BLOCK                             0
LOT                               0
ZIP CODE                          0
RESIDENTIAL UNITS                 0
COMMERCIAL UNITS                  0
TOTAL UNITS                       0
LAND SQUARE FEET                  0
GROSS SQUARE FEET                 0
YEAR BUILT                        0
TAX CLASS AT TIME OF SALE         0
BUILDING CLASS AT TIME OF SALE    0
SALE PRICE                        0
dtype: int64

Columnn and and sum of zero values:
BOROUGH : 0
BLOCK : 0
LOT : 0
ZIP CODE : 391
RESIDENTIAL UNITS : 7994
COMMERCIAL UNITS : 44222
TOTAL UNITS : 5181
LAND SQUARE FEET : 0
GROSS SQUARE FEET : 0
YEAR BUILT : 2911
TAX CLASS AT TIME OF SALE : 0
BUILDING CLASS AT TIME OF SALE : 641
SALE PRICE : 0


In [230]:
print(f"Finally: Rows: {len(cleaned_df)}, Cols: {len(cleaned_df.columns)}")

Finally: Rows: 45317, Cols: 13


In [231]:
for col in cleaned_df.columns.values:
    print(f"{col} : {sum(cleaned_df[col] == ' -  ')}")

BOROUGH : 0
BLOCK : 0
LOT : 0
ZIP CODE : 0
RESIDENTIAL UNITS : 0
COMMERCIAL UNITS : 0
TOTAL UNITS : 0
LAND SQUARE FEET : 0
GROSS SQUARE FEET : 0
YEAR BUILT : 0
TAX CLASS AT TIME OF SALE : 0
BUILDING CLASS AT TIME OF SALE : 0
SALE PRICE : 0


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [232]:
cleaned_df["LAND SQUARE FEET"] = cleaned_df['LAND SQUARE FEET'].astype(int)

In [233]:
cleaned_df.to_csv(os.path.join(PATH ,OUTPUT_FILE),index=False)